A continuacion se procede a buscar inconsistencias entre los usuarios del sistema de gestion y los usuarios del sistema de facturacion

In [ ]:
#Montar la unidad de drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Importa el módulo numpy
import numpy as np
#Importa el modulo de pandas ycrea dataframes a partir de los usuarios del foroptic
import pandas as pd
#Clientes A
#usuarios inscritos en la plataforma en la nube(Facturacion) en el servidor A
foroptic_A = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/clientes (A).xlsx')
#Clientes B
#usuarios inscritos en la plataforma en la nube(Facturacion) en el servidor B
foroptic_B = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/clientes (B).xlsx')

In [ ]:
#Crea un dataframe con los usuarios en la base de datos del Radius(Servidor para la gestion del servicio de Hibrid Fiber Coaxial)
bdd_DMA = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/rm_users.csv')
#Crea un dataframe con los usuarios en la base de datos del U2000(Servidor para la gestion del servicio de FTTH-Fiber To The Home)
bdd_u2000 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/BBDD_u2000.xlsx')

In [ ]:
#Crea dataframes a partir de los usuarios de las dos OLTs
usuarios_olt_ppl = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Planes_GPON_OLT_Principal.xlsx')
usuarios_olt_bello = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Planes_GPON_OLT_Bello.xlsx')

Los planes de internet asignados a cada usuario tienen distintos nombres para un mismo plan, para efectos de unificar los planes de internet es necesario mostrar de forma clara la informacion de los clientes de fibra optica

In [ ]:
#Une los usuarios de las dos OLTs en un solo dataframe obteniendo el total de los usuarios de fibra optica
usuarios_GPON = pd.concat([usuarios_olt_bello, usuarios_olt_ppl], ignore_index=True)
print(usuarios_GPON)

      Unnamed: 0                                 Name  Alias Connection Type  \
0              0  33/0_1_3/17/1/Multi-Service VLAN/33  73303        LAN-GPON   
1              1  33/0_1_0/45/1/Multi-Service VLAN/33   8175        LAN-GPON   
2              2  33/0_1_0/41/1/Multi-Service VLAN/33  10561        LAN-GPON   
3              3  33/0_1_1/74/1/Multi-Service VLAN/33   8192        LAN-GPON   
4              4  33/0_1_1/14/1/Multi-Service VLAN/33   8181        LAN-GPON   
...          ...                                  ...    ...             ...   
3655        2700  24/0_6_0/36/1/Multi-Service VLAN/24  11973        LAN-GPON   
3656        2701  24/0_3_8/21/1/Multi-Service VLAN/24  92295        LAN-GPON   
3657        2702  24/0_6_0/37/1/Multi-Service VLAN/24   1922        LAN-GPON   
3658        2703  24/0_5_0/65/1/Multi-Service VLAN/24   8166        LAN-GPON   
3659        2704  24/0_6_1/67/1/Multi-Service VLAN/24  11976        LAN-GPON   

                          Interface Inf

In [ ]:
#En la columna Name es ingresado el numero de contrato del usuario
#Alias : caja y puerto en la cual esta el usuario
usuarios_GPON.columns

Index(['Unnamed: 0', 'Name', 'Alias', 'Connection Type',
       'Interface Information', 'Service Type', 'Service Para',
       'Upstream Traffic Profile', 'Downstream Traffic Profile', 'VLAN ID',
       'Inner VLAN ID', 'Max. Learnable MAC Addresses', 'Frame', 'Slot',
       'Port', 'ONTID', 'Contrato', 'Serial'],
      dtype='object')

In [ ]:
#Elimina la informacion redundante de la velocidad de bajada
import re
usuarios_GPON['Downstream Traffic Profile'] = usuarios_GPON['Downstream Traffic Profile'].apply(lambda x: re.sub(r'[^0-9]', '', x))

In [ ]:
#Elimina la informacion redundante de la velocidad de subida
usuarios_GPON['Upstream Traffic Profile'] = usuarios_GPON['Upstream Traffic Profile'].apply(lambda x: re.sub(r'[^0-9]', '', x))

In [ ]:
#Se verifica que la velocidad de bajada solo contenga el numero de Mbps asignadas al usuario
usuarios_GPON['Downstream Traffic Profile'].unique()

array(['25', '15', '50', '100', '30', '10', '20', '200', '80', '5', '60',
       '150'], dtype=object)

In [ ]:
#Se verifica que la velocidad de subida solo contenga el numero de Mbps asignadas al usuario
usuarios_GPON['Downstream Traffic Profile'].unique()

array(['25', '15', '50', '100', '30', '10', '20', '200', '80', '5', '60',
       '150'], dtype=object)

In [ ]:
#Indicar la unidad de medida
usuarios_GPON['Downstream Traffic Profile'] = usuarios_GPON['Downstream Traffic Profile'].apply(lambda x: str(x) + 'MB')

In [ ]:
#Indicar la unidad de medida
usuarios_GPON['Upstream Traffic Profile'] = usuarios_GPON['Upstream Traffic Profile'].apply(lambda x: str(x) + 'MB')

In [ ]:
#Elimina las columnas innecesarias del dataframe
columnas_a_conservar = ['Contrato', 'Upstream Traffic Profile', 'Downstream Traffic Profile', 'Serial']
usuarios_GPON = usuarios_GPON[columnas_a_conservar]

In [ ]:
usuarios_GPON = usuarios_GPON.rename(columns={'Downstream Traffic Profile': 'Downstream Traffic Profile (Mb)'})

In [ ]:
usuarios_GPON = usuarios_GPON.rename(columns={'Upstream Traffic Profile': 'Upstream Traffic Profile (Mb)'})

In [ ]:
usuarios_GPON.to_excel('Usuarios_GPON.xlsx')

Ahora es necesario buscar inconsistencias entre la base de datos del sistema de gestion y la base de datos del sistema en la nube de facturacion

In [ ]:
#Une en un solo dataframe los dos foroptics (Sistemas de facturacion)
foroptic_A_y_B = pd.concat([foroptic_A, foroptic_B], ignore_index=True)

In [ ]:
#pasa la mac a tipo string para poder hacer comparaciones
foroptic_A_y_B['Mac']=foroptic_A_y_B['Mac'].astype(str)

In [ ]:
foroptic_A_y_B.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19608 entries, 0 to 19607
Data columns (total 35 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Tercero          19608 non-null  object        
 1   Ciudad           19608 non-null  object        
 2   Barrio           19608 non-null  object        
 3   Dirección        19608 non-null  object        
 4   Nombre(s)        19608 non-null  object        
 5   Apellido(s)      19608 non-null  object        
 6   Teléfonos        19564 non-null  object        
 7   Contr            19608 non-null  object        
 8   Estr             19608 non-null  int64         
 9   Doc              19608 non-null  object        
 10  # Doc            19607 non-null  object        
 11  Observaciones    14248 non-null  object        
 12  Est              19608 non-null  object        
 13  $ mes            19608 non-null  int64         
 14  F creac          18194 non-null  datet

In [ ]:
#Comprobar el tipo de dato de la columna mac
foroptic_A_y_B[foroptic_A_y_B['Contr'] == 8942].Mac

7126    0418D608A6F8
Name: Mac, dtype: object

In [ ]:
foroptic_A_y_B[foroptic_A_y_B['Mac'] == '602AD08F5674']

,Tercero,Ciudad,Barrio,Dirección,Nombre(s),Apellido(s),Teléfonos,Contr,Estr,Doc,...,Servicio,Cod paq,Paquete,Cod seg,Segmento,Cod tipo tec,Tipo tecnología,Downstream,Upstream,Mac
3639,HERNANDO GUTIERREZ,POP,JUNIN BAJO,IN IN LT 26,JOSE LUIS,COBO MOSQUERA,3152533457,5808,2,C.C.,...,G25 MB CB,1.0,Internet fijo,102,Residencial - Estrato 2,108,Fiber to the home (FTTH),25,15.0,602AD08F5674


In [ ]:

#En la base de datos de los usuarios de HFC cambia el '1' por 'activo', '0' por 'deshabilitado'
nombre_columna = "enableuser"
valor_a_cambiar_activo = 1
valor_a_cambiar_desactivado = 0

mask_1 = bdd_DMA[nombre_columna] == valor_a_cambiar_activo
mask_0 = bdd_DMA[nombre_columna] == valor_a_cambiar_desactivado
bdd_DMA.loc[mask_1, nombre_columna] = 'ACTIVO'
bdd_DMA.loc[mask_0, nombre_columna] = 'DESHABILITADO'
print(bdd_DMA['enableuser'])

0              ACTIVO
1              ACTIVO
2              ACTIVO
3              ACTIVO
4       DESHABILITADO
            ...      
3414           ACTIVO
3415           ACTIVO
3416           ACTIVO
3417           ACTIVO
3418           ACTIVO
Name: enableuser, Length: 3419, dtype: object


In [ ]:
#Cambia el tipo de equipo
#Cambia el '1' por 'inhalambrico', 6' por 'cable modem'
nombre_columna = "acctype"
valor_a_cambiar_cable_modem = 6
valor_a_cambiar_inhalambrico = 1

mask_1 = bdd_DMA[nombre_columna] == valor_a_cambiar_cable_modem
mask_0 = bdd_DMA[nombre_columna] == valor_a_cambiar_inhalambrico
bdd_DMA.loc[mask_1, nombre_columna] = 'Cable modem'
bdd_DMA.loc[mask_0, nombre_columna] = 'inhalambrico'
print(bdd_DMA['acctype'])

0        Cable modem
1        Cable modem
2        Cable modem
3        Cable modem
4       inhalambrico
            ...     
3414     Cable modem
3415     Cable modem
3416     Cable modem
3417     Cable modem
3418     Cable modem
Name: acctype, Length: 3419, dtype: object


In [ ]:
#Intercambia los valores de las MAC del contrato por los valores de contractid en los usuarios de inhalambrico
mask = bdd_DMA['username'].apply(lambda x: any(c.isalpha() for c in str(x)))  # Verifica si hay letras en username

# Intercambia los valores de username y columna2 solo si la máscara es True
bdd_DMA.loc[mask, ['username', 'contractid']] = bdd_DMA.loc[mask, ['contractid', 'username']].values

In [ ]:
#Guarda los usuarios sin contrato en la base de datos del DMA
mask_nan = bdd_DMA['username'].isna()
mask_letras = bdd_DMA['username'].apply(lambda x: any(c.isalpha() for c in str(x)))
mask_resultado = mask_nan | mask_letras

# Filtrar las filas que cumplen la condición y guardarlas en un nuevo DataFrame
df_filtrado = bdd_DMA[mask_resultado]

In [ ]:
#Cambia de lugar las columnas username y contractid de los usuarios sin contrato
temp = df_filtrado['username'].copy()
df_filtrado['username'] = df_filtrado['contractid']
df_filtrado['contractid'] = temp

<ipython-input-21-2b7c07d6885b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['username'] = df_filtrado['contractid']
<ipython-input-21-2b7c07d6885b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['contractid'] = temp


In [ ]:
#Guarda los valores de los usuarios que aparecen sin un numero de contrato en el DMA
df_filtrado.to_excel('DMA_sin_contrato.xlsx')
file_path = '/content/DMA_sin_contrato.xlsx'
excel_dest_path = '/content/drive/MyDrive/Colab Notebooks/DMA_sin_contrato.xlsx'
!mv "$file_path" "$excel_dest_path"

In [ ]:
#Eliminar los valores vacíos, y los que contengan letras
mask_nan = bdd_DMA['username'].isna()
mask_letras = bdd_DMA['username'].apply(lambda x: any(c.isalpha() for c in str(x)))
mask_resultado = mask_nan | mask_letras
bdd_DMA.drop(bdd_DMA[mask_resultado].index, inplace=True)
# Filtrar y quedarte con las filas que NO cumplen la condiciónbdd_DMA_sin_letras_nan =bdd_DMA[~mask_resultado]

In [ ]:
#Corregir un espacio en la columna contrato
bdd_DMA['username'].loc[bdd_DMA['username']=='55010282 ']='55010282'
bdd_DMA

<ipython-input-24-3a846b62ce99>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bdd_DMA['username'].loc[bdd_DMA['username']=='55010282 ']='55010282'


,username,password,groupid,enableuser,uplimit,downlimit,comblimit,firstname,lastname,company,...,verifysentnum,verifymobile,contractid,contractvalid,actcode,pswactsmsnum,alertemail,alertsms,lang,lastlogoff
0,0000,a20e7c2bc0f7f350ce7b10ed36c732fc,28,ACTIVO,0,0,0,JOSE MIGUEL,CHILITO,CABLE CAUCA S.A,...,0,NaN,0000,0000-00-00,NaN,0,1,1,English,NaN
1,0002,9bb33b7aae5a143efa325c6b6b43e68c,34,ACTIVO,0,0,0,PRUEBA PILAQUINGA,NaN,CABLE CAUCA S.A,...,0,NaN,0002,0000-00-00,NaN,0,1,1,English,NaN
2,0003,4e8688720e6140d8ce3323a827ca4afe,34,ACTIVO,0,0,0,PRUEBA BONILLA,NaN,CABLE CAUCA S.A,...,0,NaN,0003,0000-00-00,NaN,0,1,1,English,NaN
3,0004,4f2eb2601322cd165b9df6a300bf4b47,197,ACTIVO,0,0,0,FRANKLYN,SEGURA - CASA GARZAS,CABLE CAUCA S.A,...,0,NaN,0004,0000-00-00,NaN,0,1,1,English,NaN
4,8481,3825c3cc34c3126da486da7b573949ec,129,DESHABILITADO,0,0,0,FLOR ISABEL,PAME DIAZ,Cable Cauca S.A,...,0,NaN,00:27:22:7c:55:1b,0000-00-00,NaN,0,1,1,Spanish,2022-01-18 08:27:15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3391,1991,fc6bb94329914935499d1aef2bd4d7f8,33,ACTIVO,0,0,0,DAGNOVER,MAMIAN PALECHOR,Cable Cauca S.A,...,0,NaN,dc:9f:db:88:5a:86,0000-00-00,NaN,0,1,1,Spanish,2023-09-05 11:50:11
3392,8627,d8ec1f6360a417f1b8eedc08261ff3b3,33,ACTIVO,0,0,0,EDWIN,ALEGRIA,Cable Cauca S.A,...,0,NaN,dc:9f:db:88:5d:40,0000-00-00,NaN,0,1,1,Spanish,2023-09-06 00:32:55
3393,55010282,a089ad87adc951a62eac3d9083507b45,74,ACTIVO,0,0,0,YADIRA,CAMPO BRAVO,CABLECAUCA S.A,...,0,NaN,dc:9f:db:88:7b:1f,0000-00-00,NaN,0,1,1,English,2023-09-05 11:48:53
3394,5166,b305983918e9da5f7a03b00cc36f5dd2,33,ACTIVO,0,0,0,SANDRA SOFIA,MEDINA ROJAS,Cable Cauca S.A,...,0,NaN,dc:9f:db:e4:6e:e5,0000-00-00,NaN,0,1,1,Spanish,2023-09-03 23:28:18


In [ ]:
#Elimina un contrato con letra en vez de numero
foroptic_A_y_B.drop(foroptic_A_y_B[foroptic_A_y_B['Contr'] =='55I00595'].index, inplace=True)


In [ ]:
#Pasar a enteros las columnas de contratos
bdd_DMA['username'] = bdd_DMA['username'].astype(int)
foroptic_A_y_B['Contr'] = foroptic_A_y_B['Contr'].astype(int)

In [ ]:
#Comprueba los valores de la columna servicio
foroptic_A_y_B['Servicio'].unique()

array(['G25 MB CB', 'G15 MB', 'G10 MB CB', 'G15 MB CB', 'G50 MB CB',
       'G25 MB', '@ 5 MB CB', 'G30 MB CB', 'Servicio de TV', 'PHogar 1MB',
       '@ 4 MB CB', '@ 10 MB CB', '@ 15 MB CB', '@ 10 MB', '@ 8 MB CB',
       'G80 MB CB', 'G100 MB CB', '@ 3 MB CB', 'G10 MB', 'G100 MB',
       '@ 6 MB CB', '@ 1 MB CB', 'G5 MB CB', 'G50 MB', 'W 3MB CB UB',
       'G150 MB CB', 'G20 MB CB', '@ 2 MB CB', 'G30 MB', '@ 1 MB',
       'W 1MB CB UB', '@ 12 MB CB', '@ 5 MB', '@ 9 MB CB', 'W 5MB CB UB',
       'G20 MB', 'G5 MB', 'D 20MB', 'W 2MB CB UB', 'W 2MB', '@ 9 MB',
       'W 1MB', 'W 3MB', 'W 10MB CB UB', 'W 5MB', '@ 2 MB', 'G200 MB',
       'G200 MB CB', '@ 5 MB CB COR', 'W 4MB CB UB', 'W 20MB'],
      dtype=object)

In [ ]:
#Mantiene los usuarios de Cable modem e inhalambrico descartando los usuarios de fibra optica
foropti = foroptic_A_y_B[foroptic_A_y_B['Servicio'].str.startswith('@')]
forop = foroptic_A_y_B[foroptic_A_y_B['Servicio'].str.startswith('W')]
foroptic = pd.concat([foropti, forop], ignore_index=True)

In [ ]:
#Comprueba el filtro de cable modem e inhalambrico
foroptic['Servicio'].unique()

array(['@ 5 MB CB', '@ 4 MB CB', '@ 10 MB CB', '@ 15 MB CB', '@ 10 MB',
       '@ 8 MB CB', '@ 3 MB CB', '@ 6 MB CB', '@ 1 MB CB', '@ 2 MB CB',
       '@ 1 MB', '@ 12 MB CB', '@ 5 MB', '@ 9 MB CB', '@ 9 MB', '@ 2 MB',
       '@ 5 MB CB COR', 'W 3MB CB UB', 'W 1MB CB UB', 'W 5MB CB UB',
       'W 2MB CB UB', 'W 2MB', 'W 1MB', 'W 3MB', 'W 10MB CB UB', 'W 5MB',
       'W 4MB CB UB', 'W 20MB'], dtype=object)

In [ ]:
#Compara los contratos en el dma con los contratos en el foroptic y obtiene los valores del dma(Gestion) no encontrados en foroptic(Facturacion)
columna_referencia = foroptic['Contr']
columna_a_verificar= bdd_DMA['username']
valores_no_encontrados = columna_a_verificar[~columna_a_verificar.isin(columna_referencia)]
print(valores_no_encontrados)

0           0
1           2
2           3
3           4
7        6254
        ...  
3387    11299
3388     5449
3389    16349
3394     5166
3395     5386
Name: username, Length: 173, dtype: int64


In [ ]:
#Crea un dataframe con los contratos no encontrados
data = {'Contrato': valores_no_encontrados}
df_nuevos_valores = pd.DataFrame(data)

print(df_nuevos_valores)

      Contrato
0            0
1            2
2            3
3            4
7         6254
...        ...
3387     11299
3388      5449
3389     16349
3394      5166
3395      5386

[173 rows x 1 columns]


In [ ]:
bdd_DMA.columns

Index(['username', 'password', 'groupid', 'enableuser', 'uplimit', 'downlimit',
       'comblimit', 'firstname', 'lastname', 'company', 'phone', 'mobile',
       'address', 'city', 'zip', 'country', 'state', 'comment', 'gpslat',
       'gpslong', 'mac', 'usemacauth', 'expiration', 'uptimelimit', 'srvid',
       'staticipcm', 'staticipcpe', 'ipmodecm', 'ipmodecpe', 'poolidcm',
       'poolidcpe', 'createdon', 'acctype', 'credits', 'cardfails',
       'createdby', 'owner', 'taxid', 'cnic', 'email', 'maccm', 'custattr',
       'warningsent', 'verifycode', 'verified', 'selfreg', 'verifyfails',
       'verifysentnum', 'verifymobile', 'contractid', 'contractvalid',
       'actcode', 'pswactsmsnum', 'alertemail', 'alertsms', 'lang',
       'lastlogoff'],
      dtype='object')

In [ ]:
# Comparar y agregar velocidad de bajada al dataframe de los valores no encontrados
def agregar_valor(row):
    matching_row = bdd_DMA[
        (bdd_DMA['username'] == row['Contrato'])
    ]
    if not matching_row.empty:
        return matching_row.iloc[0][['srvid', 'enableuser', 'firstname', 'lastname', 'acctype', 'maccm', 'contractid']]  # Cambiar por la columna que desees
    return None

df_nuevos_valores[['Velocidad (Mb)', 'Estado', 'Nombre', 'Apellido', 'Tecnología', 'MAC', 'MAC-WI-FI']] = df_nuevos_valores.apply(agregar_valor, axis=1)

In [ ]:
df_nuevos_valores.columns

Index(['Contrato', 'Velocidad (Mb)', 'Estado', 'Nombre', 'Apellido',
       'Tecnología', 'MAC', 'MAC-WI-FI'],
      dtype='object')

In [ ]:
#pasar las mac de formato dc:9f:db:e4:6e:e5 a dc9fdbe46ee5 (asi estan registradas en el sistema de facturacion)
df_nuevos_valores['MAC'] = df_nuevos_valores['MAC'].str.replace(':', '')
df_nuevos_valores['MAC-WI-FI'] = df_nuevos_valores['MAC-WI-FI'].str.replace(':', '')


In [ ]:
#junta las mac de los equipos de cable modem con las mac de los equipos del servicio inalambrico
df_nuevos_valores['MAC'].fillna(df_nuevos_valores['MAC-WI-FI'], inplace=True)

In [ ]:
df_nuevos_valores.drop(columns=['MAC-WI-FI'], inplace=True)

In [ ]:
df_nuevos_valores['MAC'] = df_nuevos_valores['MAC'].str.upper()  # Convertir a mayúsculas
foroptic_A_y_B['Mac'] = foroptic_A_y_B['Mac'].str.upper()

In [ ]:
foroptic_A_y_B.columns

Index(['Tercero', 'Ciudad', 'Barrio', 'Dirección', 'Nombre(s)', 'Apellido(s)',
       'Teléfonos', 'Contr', 'Estr', 'Doc', '# Doc', 'Observaciones', 'Est',
       '$ mes', 'F creac', 'F prim Act', 'F Ult DX', 'F Ult P', '$ Ult P',
       'Saldo', 'Mora', 'Clase', 'Cod Tar', 'Tarifa', 'Cod Serv', 'Servicio',
       'Cod paq', 'Paquete', 'Cod seg', 'Segmento', 'Cod tipo tec',
       'Tipo tecnología', 'Downstream', 'Upstream', 'Mac'],
      dtype='object')

In [ ]:
#Para comparar es necesario que los datos tengan el mismo tipo de dato
foroptic_A_y_B['Contr'].astype(int)

0            8694
1           10310
2            5889
3            8865
4            8884
           ...   
19603    55016878
19604       91202
19605       89062
19606       88543
19607      557209
Name: Contr, Length: 19607, dtype: int64

In [ ]:
foroptic_A_y_B['Contr'].info()

<class 'pandas.core.series.Series'>
Int64Index: 19607 entries, 0 to 19607
Series name: Contr
Non-Null Count  Dtype
--------------  -----
19607 non-null  int64
dtypes: int64(1)
memory usage: 306.4 KB


In [ ]:
#Comparar por mac en las dos bases de datos y extraer la columna con la informacion de contrato
def agregar_valor(row):
    matching_row = foroptic_A_y_B[
        (foroptic_A_y_B['Mac'] == row['MAC'])
    ]
    if not matching_row.empty:
        return matching_row.iloc[0]['Contr']  # Cambiar por la columna que desees
    return None

df_nuevos_valores['Contrato-MAC en Foroptic'] = df_nuevos_valores.apply(agregar_valor, axis=1)

In [ ]:
df_nuevos_valores['Contrato-MAC en Foroptic'].info()

<class 'pandas.core.series.Series'>
Int64Index: 173 entries, 0 to 3395
Series name: Contrato-MAC en Foroptic
Non-Null Count  Dtype  
--------------  -----  
45 non-null     float64
dtypes: float64(1)
memory usage: 2.7 KB


In [ ]:
#Comparar por mac en las dos bases de datos y extraer la columna con la informacion del tipo de tecnologia
def agregar_valor(row):
    matching_row = foroptic_A_y_B[
        (foroptic_A_y_B['Mac'] == row['MAC'])
    ]
    if not matching_row.empty:
        return matching_row.iloc[0]['Servicio']  # Cambiar por la columna que desees
    return None

df_nuevos_valores['Tecnología-MAC en Foroptic'] = df_nuevos_valores.apply(agregar_valor, axis=1)

In [ ]:
#Comparar por contrato en las dos bases de datos y extraer la columna con la informacion de la mac de los equipos instalados
def agregar_valor(row):
    matching_row = foroptic_A_y_B[
        (foroptic_A_y_B['Contr'] == row['Contrato'])
    ]
    if not matching_row.empty:
        return matching_row.iloc[0]['Mac']  # Cambiar por la columna que desees
    return None

df_nuevos_valores['MAC-contrato-foroptic'] = df_nuevos_valores.apply(agregar_valor, axis=1)

In [ ]:
#Comparar por contrato en las dos bases de datos y extraer la columna con la informacion del tipo de tecnologia
def agregar_valor(row):
    matching_row = foroptic_A_y_B[
        (foroptic_A_y_B['Contr'] == row['Contrato'])
    ]
    if not matching_row.empty:
        return matching_row.iloc[0]['Servicio']  # Cambiar por la columna que desees
    return None

df_nuevos_valores['Tecnología-contrato en Foroptic'] = df_nuevos_valores.apply(agregar_valor, axis=1)

Ahora es necesario buscar los usuarios de HFC no encontrados en la base de datos de los usuarios de FTTH por si existe un error en el tipo de tecnologia
  

In [ ]:
#adapta la informacion de la base de datos del u2000(Gestion de usuarios de fibra optica) a la informacion del foroptic
#El serial de los equipos de usuario (ONT) no tienen caracteres especiales y tienen 8 caracteres en vez de 12
bdd_u2000['Sn'] = bdd_u2000['Sn'].str.replace(r'\(.*\)', '')

<ipython-input-47-0f5a644a8c13>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  bdd_u2000['Sn'] = bdd_u2000['Sn'].str.replace(r'\(.*\)', '')


In [ ]:
bdd_u2000['Sn'] = bdd_u2000['Sn'].str[4:]  # Eliminar los primeros cuatro caracteres

In [ ]:
df_nuevos_valores.columns

Index(['Contrato', 'Velocidad (Mb)', 'Estado', 'Nombre', 'Apellido',
       'Tecnología', 'MAC', 'Contrato-MAC en Foroptic',
       'Tecnología-MAC en Foroptic', 'MAC-contrato-foroptic',
       'Tecnología-contrato en Foroptic'],
      dtype='object')

In [ ]:
#Comparar por serial en las dos bases de datos y extraer la columna con la informacion del numero de contrato
def agregar_valor(row):
    matching_row = bdd_u2000[
        (bdd_u2000['Sn'] == row['MAC-contrato-foroptic'])
    ]
    if not matching_row.empty:
        return matching_row.iloc[0]['Name']  # Cambiar por la columna que desees
    return None

df_nuevos_valores['Contrato-MAC en U2000'] = df_nuevos_valores.apply(agregar_valor, axis=1)

In [ ]:
#Elimina cualquier numero de contrato que contenga una letra
mask_letras = bdd_u2000['Name'].apply(lambda x: any(c.isalpha() for c in str(x)))
bdd_u2000.drop(bdd_u2000[mask_letras].index, inplace=True)

In [ ]:
bdd_u2000['Name'] = bdd_u2000['Name'].astype(int)

In [ ]:
df_nuevos_valores.reset_index(inplace=True, drop=True)

In [ ]:
#La velocidad de bajada de los planes de internet se muestra con un identificador
df_nuevos_valores['Velocidad (Mb)'].unique()

array([53, 37, 42, 47, 39, 38, 48, 50, 49, 41, 40, 52])

In [ ]:
#Cambia el identificador por el nombre de los planes de usuario.
nombre_columna = 'Velocidad (Mb)'
valor_a_cambiar_37 = 37
valor_a_cambiar_38 = 38
valor_a_cambiar_39 = 39
valor_a_cambiar_40 = 40
valor_a_cambiar_41 = 41
valor_a_cambiar_42 = 42
valor_a_cambiar_47 = 47
valor_a_cambiar_48 = 48
valor_a_cambiar_49 = 49
valor_a_cambiar_50 = 50
valor_a_cambiar_52 = 52
valor_a_cambiar_53 = 53

mask_1 = df_nuevos_valores[nombre_columna] == valor_a_cambiar_37
mask_2 = df_nuevos_valores[nombre_columna] == valor_a_cambiar_38
mask_3 = df_nuevos_valores[nombre_columna] == valor_a_cambiar_39
mask_4 = df_nuevos_valores[nombre_columna] == valor_a_cambiar_40
mask_5 = df_nuevos_valores[nombre_columna] == valor_a_cambiar_41
mask_6 = df_nuevos_valores[nombre_columna] == valor_a_cambiar_42
mask_7 = df_nuevos_valores[nombre_columna] == valor_a_cambiar_47
mask_8 = df_nuevos_valores[nombre_columna] == valor_a_cambiar_48
mask_9 = df_nuevos_valores[nombre_columna] == valor_a_cambiar_49
mask_10 = df_nuevos_valores[nombre_columna] == valor_a_cambiar_50
mask_11 = df_nuevos_valores[nombre_columna] == valor_a_cambiar_52
mask_12 = df_nuevos_valores[nombre_columna] == valor_a_cambiar_53

df_nuevos_valores.loc[mask_1, nombre_columna] = '5 Mb'
df_nuevos_valores.loc[mask_2, nombre_columna] = '6 Mb'
df_nuevos_valores.loc[mask_3, nombre_columna] = '8 Mb'
df_nuevos_valores.loc[mask_4, nombre_columna] = '9 Mb'
df_nuevos_valores.loc[mask_5, nombre_columna] = '10 Mb'
df_nuevos_valores.loc[mask_6, nombre_columna] = '12 Mb'
df_nuevos_valores.loc[mask_7, nombre_columna] = 'Wireless 5 Mb'
df_nuevos_valores.loc[mask_8, nombre_columna] = 'Wireless 6 Mb'
df_nuevos_valores.loc[mask_9, nombre_columna] = 'Wireless 8 Mb'
df_nuevos_valores.loc[mask_10, nombre_columna] = 'Wireless 10 Mb'
df_nuevos_valores.loc[mask_11, nombre_columna] = '20 Mb'
df_nuevos_valores.loc[mask_12, nombre_columna] = '15 Mb'
print(df_nuevos_valores['Velocidad (Mb)'])

0               15 Mb
1                5 Mb
2               15 Mb
3               12 Mb
4       Wireless 5 Mb
            ...      
168     Wireless 5 Mb
169     Wireless 5 Mb
170     Wireless 5 Mb
171     Wireless 6 Mb
172    Wireless 10 Mb
Name: Velocidad (Mb), Length: 173, dtype: object


In [ ]:
#Guarda en Drive un archivo excel con los valores no encontrados.
df_nuevos_valores.to_excel('Valores del DMA no encontrados en el foroptic.xlsx')
file_path = '/content/Valores del DMA no encontrados en el foroptic.xlsx'
excel_dest_path = '/content/drive/MyDrive/Colab Notebooks/Valores del DMA no encontrados en el foroptic.xlsx'
!mv "$file_path" "$excel_dest_path"

In [ ]:
usuarios_GPON.to_excel('Usuarios_GPON.xlsx')